In [2]:
# --- Install Poppler and other dependencies for UnstructuredPDFLoader ---
# Install poppler-utils for pdfinfo and other PDF processing tools
!sudo apt-get update
!sudo apt-get install -y poppler-utils

# Install pytesseract and tesseract-ocr for OCR capabilities (optional, but good for scanned PDFs)
!pip install pytesseract
!sudo apt-get install -y tesseract-ocr

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,266 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,103 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64

In [3]:
pip install langchain-community transformers accelerate bitsandbytes faiss-cpu sentence-transformers unstructured pypdf unstructured-pytesseract pi-heif pdfminer.six langchain-huggingface unstructured-inference pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [30]:
import os
token =""
os.environ["HUGGINGFACEHUB_API_TOKEN"] = token

In [26]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline,BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import torch

In [12]:
PDF_PATH = "/content/AI Training Document (1) (1) (1).pdf"
CHUNK_SIZE = 1500
CHUNK_OVERLAP = 200
TOP_K_RETRIEVAL = 5

In [13]:
 loader = UnstructuredPDFLoader(PDF_PATH)
 documents = loader.load()

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    is_separator_regex=False,
)
docs = text_splitter.split_documents(documents)

In [16]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-16-3409896792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
db = FAISS.from_documents(docs, embeddings)

retriever = db.as_retriever(search_kwargs={"k": TOP_K_RETRIEVAL})

In [18]:
from huggingface_hub import login

# You'll be prompted to paste your token (starts with `hf_...`)
login()


In [19]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    trust_remote_code=True,
    load_in_8bit=True,  # Use 8-bit quantization for lower memory usage
    device_map="auto" # Automatically maps model layers to available devices
)

# Create a text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512, # Max tokens the LLM will generate in response
    do_sample=True,
    temperature=0.7,    # Controls randomness: lower for more deterministic, higher for more creative
    top_k=50,           # Limits the sampling to the top k most likely tokens
    top_p=0.95,         # Nucleus sampling: only tokens with cumulative probability p are considered
)

# Wrap the Hugging Face pipeline in LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


In [20]:
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Given the following context, please provide a concise answer to the question.
Your response MUST contain only the answer and nothing else. Do not include the question, context, or any introductory/concluding phrases.

Context:
{context}

Question: {question}

Answer:
""".strip()
)

# Construct the RAG chain using LCEL
rag_chain = (
    # Step 1: Retrieve context based on the input question
    {"context": retriever, "question": RunnablePassthrough()}
    # Step 2: Format the retrieved context and question into the prompt template
    | prompt_template
    # Step 3: Pass the formatted prompt to the LLM for generation
    | llm
    # Step 4: Parse the LLM's output into a string, stripping any leading/trailing whitespace
    | StrOutputParser()
)

In [22]:
 query="give the title of the first chapter or first page"
 response = rag_chain.invoke(query)

  # Post-process the response to ensure only the answer is returned
 answer = response.strip()
 if answer.lower().startswith("answer:"):
  answer = answer[len("answer:"):].strip()

 print(f"\n🧠 Answer: {answer}\n")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🧠 Answer: Given the following context, please provide a concise answer to the question.
Your response MUST contain only the answer and nothing else. Do not include the question, context, or any introductory/concluding phrases.

Context:
[Document(id='4da66fe6-bd0b-4aa3-8500-c6b72adbf87b', metadata={'source': '/content/AI Training Document (1) (1) (1).pdf'}, page_content="You are responsible for reading the full item listing before making a bid or offer, buying, or committing to buy;\n\nYou enter into a legally binding contract to purchase an item when you buy the item, commit to buy the item, your offer for the item is accepted, you have the winning bid for the item, or your bid for the item is otherwise accepted, regardless of when payment is due or received unless the transaction terms state that you are required to pay for the item before a binding contract is formed;\n\ne For motor vehicles and real estate, a bid or offer initiates a non-binding transaction representing a buyer's 